In [ ]:
from datetime import date
from itertools import product

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

from app.model import DispatchConfig
from run_dispatch import run_dispatch

In [ ]:
DATES = pd.date_range(
    start="2024-04-01",
    end="2024-04-30",
    inclusive="both"
)
# DATES = [
#     pd.Timestamp("2024-02-24"),
#     pd.Timestamp("2024-03-27"),
#     pd.Timestamp("2024-02-05"),
#     pd.Timestamp("2024-08-11"),
#     pd.Timestamp("2024-04-23"),
#     pd.Timestamp("2024-08-27"),
#     pd.Timestamp("2024-03-05"),
# ]
CONFIGS = [
    "preideal",
    # "bess_preideal",
    "bess_preideal_resource",
]

BESS_VALUES = {
    "BESS_1": {
        "MWh_nom": 2000,
        "hours_to_deplete": 2,
        "efficiency": 0.9,
        "min_soc": 0.05,
        "max_soc": 0.98,
        "initial_soc": 0.05,
        "charge_bid": 420000,
        "discharge_bid": 550000,
    },
}


In [ ]:

output_unstacked = pd.DataFrame()
for (date_, dispatch_type) in product(DATES,CONFIGS):
    BESS = BESS_VALUES if "bess" in dispatch_type.lower() else None
    mpo = run_dispatch(DispatchConfig(dispatch_type=dispatch_type), DISPATCH_DATE=date_.date(), show_figs=False, BESS=BESS)
    # mpo["dispatch_type"] = dispatch_typex
    output_unstacked = pd.concat([output_unstacked, mpo])
output = output_unstacked.stack().reset_index()
output.columns = ["datetime", "source", "MPO"]
output.drop_duplicates(inplace=True)

In [ ]:

fig = px.line(output, x="datetime", y="MPO", color="source", title="MPO")
fig.update_layout(
    xaxis_title="Hora",
    yaxis_title="Precio [COP/MWh]",
    # width=800,
    # height=600,
    # xaxis=dict(
    #     dtick=3_600_000,
    # ),
)
fig_2 = go.Figure()
fig_2.add_traces(
    fig.data
)
fig_2.show()

In [ ]:
output_unstacked = output.pivot(index="datetime", columns="source", values="MPO")

In [ ]:
# Group by days and apply the mean_absolute_percentage_error function
grouped = output_unstacked.groupby(output_unstacked.index.date).apply(
    lambda x: mean_absolute_percentage_error(y_pred=x["MPO preideal Modelo"], y_true=x["MPO preideal XM"])
)

# Convert the result to a DataFrame for better readability
grouped_df = pd.DataFrame(grouped, columns=["MAPE %"])
grouped_df["MAPE %"] = grouped_df["MAPE %"] * 100
grouped_df

In [ ]:
# Calculate the difference between the two columns
output_unstacked['Difference'] = output_unstacked['MPO preideal Modelo'] - output_unstacked['MPO bess_preideal_resource Modelo']

# Create an area plot with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['Difference'],
    fill='tozeroy',
    mode='none'  # No markers or lines
))

fig.update_layout(
    title='Difference between MPO preideal Modelo and MPO bess_preideal Modelo',
    xaxis_title='Datetime',
    yaxis_title='Difference [COP/MWh]'
)

fig.show()

In [ ]:
output_unstacked

In [ ]:
fig = go.Figure()

# Add line for "MPO preideal Modelo"
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['MPO preideal Modelo'],
    mode='lines',
    name='MPO preideal Modelo'
))

# Add line for "MPO bess_preideal Modelo"
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['MPO bess_preideal_resource Modelo'],
    mode='lines',
    name='MPO bess_preideal Modelo',
    fill='tonexty',
    line_color='green'
))

# Add area for positive differences
# fig.add_trace(go.Scatter(
#     x=output_unstacked.index,
#     y=output_unstacked['Difference'].apply(lambda x: x if x > 0 else 0),
#     fill='tozeroy',
#     mode='none',
#     fillcolor='rgba(0, 255, 0, 0.5)',  # Green color for positive differences
#     name='Positive Difference',
#     line_shape='hv',
#     yaxis='y2'
# ))

# # Add area for negative differences
# fig.add_trace(go.Scatter(
#     x=output_unstacked.index,
#     y=output_unstacked['Difference'].apply(lambda x: x if x < 0 else 0),
#     fill='tozeroy',
#     mode='none',
#     fillcolor='rgba(255, 0, 0, 0.5)',  # Red color for negative differences
#     name='Negative Difference',
#     line_shape='hv',
#     yaxis='y2'
# ))

fig.update_layout(
    title='MPO preideal Modelo vs MPO bess_preideal Modelo',
    xaxis_title='Datetime',
    yaxis_title='MPO [COP/MWh]',
    yaxis2=dict(
        title='Difference [COP/MWh]',
        overlaying='y',
        side='right'
    ),
    legend_title='Legend'
)

fig.show()

In [ ]:
output_unstacked.columns